In [3]:
import geopandas as gpd
import folium
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
PlanningArea: gpd.GeoDataFrame = gpd.read_file('../data/MasterPlan2019PlanningAreaBoundary.geojson')
print(PlanningArea.crs)
PlanningArea.head()

EPSG:4326


,Name,Description,geometry
0,kml_1,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((103.93208 1.30555 0, 103.93208 1.3..."
1,kml_2,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((103.72042 1.32824 0, 103.72003 1.3..."
2,kml_3,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((103.76408 1.37001 0, 103.76444 1.3..."
3,kml_4,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((103.82361 1.26018 0, 103.82362 1.2..."
4,kml_5,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((103.77445 1.39029 0, 103.77499 1.3..."


In [5]:
# Load the shapefile into a GeoDataFrame
BusStops: gpd.GeoDataFrame = gpd.read_file('../data/BusStopLocation_Jul2024/BusStop.shp')

# Ensure the GeoDataFrame is in the correct CRS (coordinate reference system)
# The .prj file indicates that the data is in SVY21, which is EPSG:3414
BusStops: gpd.GeoDataFrame = BusStops.to_crs(epsg=4326)  # Convert to WGS 84 (EPSG:4326) for Folium
BusStops.head()

,BUS_STOP_N,BUS_ROOF_N,LOC_DESC,geometry
0,65059,B12,ST ANNE'S CH,POINT (103.9013 1.39303)
1,16171,B06,YUSOF ISHAK HSE,POINT (103.77437 1.29892)
2,61101,NIL,BLK 120,POINT (103.8637 1.33564)
3,01239,B01,SULTAN PLAZA,POINT (103.86165 1.30285)
4,17269,B01,BLK 730,POINT (103.76264 1.30492)


Commented out below was using the mrt station data from LTA itself, which had many issues. Use the URA data from data.gov instead

In [121]:
# def fix_invalid_geometries(gdf):
#     gdf['geometry'] = gdf['geometry'].apply(lambda geom: geom.buffer(0) if not geom.is_valid else geom)
#     return gdf

# import pyogrio

# # dataset has a problem and this solves it
# pyogrio.set_gdal_config_options({"OGR_GEOMETRY_ACCEPT_UNCLOSED_RING": "OFF"})

# Stations: gpd.GeoDataFrame = gpd.read_file('../data/TrainStation_Jul2024/RapidTransitSystemStation.shp')
# Stations = fix_invalid_geometries(Stations)
# Stations.to_crs(epsg=4326, inplace=True)

# # getting the area of the stations reveals that some a unusaly big
# Stations['area'] = Stations['geometry'].to_crs(epsg=3857).area

# # get the fake stations
# Stations["is_mrt_or_lrt"] = Stations['STN_NAM_DE'].str.contains(r"MRT STATION|LRT STATION", regex=True, case=False, na=False)
# Stations.where(Stations["is_mrt_or_lrt"] == False).dropna(subset=["is_mrt_or_lrt"])[["STN_NAM_DE", "area"]]

# Stations = Stations.where(Stations["is_mrt_or_lrt"] == True).dropna(subset=["is_mrt_or_lrt"])

In [6]:
RailStations = gpd.read_file('../data/MasterPlan2019RailStationLayer.geojson')
RailLines = gpd.read_file('../data/MasterPlan2019RailLineLayer.geojson')

In [10]:
RailLines.head()

,Name,Description,geometry
0,kml_1,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (103.7365 1.35301 0, 103.73649 1...."
1,kml_2,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (103.73708 1.35164 0, 103.73768 1..."
2,kml_3,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (103.71328 1.35252 0, 103.71327 1..."
3,kml_4,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (103.7132 1.35265 0, 103.71318 1...."
4,kml_5,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (103.70767 1.34888 0, 103.70767 1..."


In [17]:
# Read the Station code file
# Load the Excel file for station details
excel_file_path = '../data/Train Station Codes and Chinese Names.xls'
train_station_data = pd.read_excel(excel_file_path)

# Drop the Chinese names column
train_station_data = train_station_data.drop(columns=['mrt_station_chinese', 'mrt_line_chinese'])

# Normalize the station names in both datasets to lowercase for joining
train_station_data['StationName'] = train_station_data['mrt_station_english'].str.upper()

# Merge station codes if a station has multiple entries (e.g., NS17/CC15)
train_station_data = train_station_data.groupby('StationName')['stn_code'].apply(lambda x: '/'.join(x)).reset_index()


print(train_station_data)

          StationName stn_code
0           ADMIRALTY     NS10
1            ALJUNIED      EW9
2          ANG MO KIO     NS16
3               BAKAU      SE3
4             BANGKIT      BP9
..                ...      ...
175  WOODLANDS SOUTH       TE3
176         WOODLEIGH     NE11
177           YEW TEE      NS5
178      YIO CHU KANG     NS15
179            YISHUN     NS13

[180 rows x 2 columns]


In [24]:
# Create a Folium map centered around the mean coordinates of the GeoDataFrame
loc_centre = (1.359394, 103.814301)
m = folium.Map(location=loc_centre, zoom_start=12)

# Add the GeoDataFrame to the map
folium.GeoJson(RailStations).add_to(m)
#folium.GeoJson(RailLines).add_to(m)

m

In [53]:
# Function to map MRT line code prefixes to corresponding colors
def get_line_color(stn_code):
    if pd.isna(stn_code):  # Handle NaN cases
        return 'gray'  # Default color for missing station code
    if stn_code.startswith('NS'):
        return 'lightred'
    elif stn_code.startswith('EW'):
        return 'green'
    elif stn_code.startswith('NE'):
        return 'purple'
    elif stn_code.startswith('CC'):
        return 'orange'
    elif stn_code.startswith('DT'):
        return 'blue'
    elif stn_code.startswith('TE'):
        return 'darkred'
    else:
        return 'gray'  # Default color if the line is not recognized

In [54]:
# Create a base folium map centered at Singapore
map_center = [1.3521, 103.8198]  # Coordinates for Singapore
mrt_map = folium.Map(location=map_center, zoom_start=12)

import re

# Normalize the station names to lowercase for both DataFrames for easier merging
train_station_data['StationName'] = train_station_data['StationName'].str.lower().replace(" station", "").str.replace(" interchange", "").str.strip()
RailStations['StationName'] = RailStations['Description'].apply(lambda desc: re.sub(r"\b(STATION|INTERCHANGE)\b", "", re.search(r'<th>NAME</th>\s*<td>(.*?)</td>', desc).group(1)).strip().lower())

# Merge the train_station_data with the GeoDataFrame on the StationName column
merged_data = pd.merge(RailStations, train_station_data, on='StationName', how='left')

# Create a base folium map centered at Singapore
map_center = [1.3521, 103.8198]  # Coordinates for Singapore
mrt_map = folium.Map(location=map_center, zoom_start=12)

# Iterate over the merged stations
for idx, row in merged_data.iterrows():
    # Extract the station code (stn_code) and other relevant details
    stn_code = row['stn_code']
    feature = row["Description"]
    grnd_level = re.search(r'<th>GRND_LEVEL</th>\s*<td>(.*?)</td>', feature).group(1)
    rail_type = re.search(r'<th>RAIL_TYPE</th>\s*<td>(.*?)</td>', feature).group(1)
    name = re.search(r'<th>NAME</th>\s*<td>(.*?)</td>', feature).group(1)
    inc_crc = re.search(r'<th>INC_CRC</th>\s*<td>(.*?)</td>', feature).group(1)
    fmel_upd_d = re.search(r'<th>FMEL_UPD_D</th>\s*<td>(.*?)</td>', feature).group(1)

    # Remove the word 'Interchange' from the station name if it exists
    cleaned_name = name.replace(" INTERCHANGE", "").replace(" STATION", "")
 

    # Build the popup text using the extracted values and the station code
    popup_text = f"""
    <div style="padding: 10px;">
    <table style="border:1px solid black;">
        <tr><th>Station Name</th><td>{cleaned_name}</td></tr>
        <tr><th>Station Code</th><td>{stn_code}</td></tr>
        <tr><th>Ground Level</th><td>{grnd_level}</td></tr>
        <tr><th>Rail Type</th><td>{rail_type}</td></tr>
        <tr><th>INC_CRC</th><td>{inc_crc}</td></tr>
        <tr><th>FMEL_UPD_D</th><td>{fmel_upd_d}</td></tr>
    </table>
    </div>
    """

    # Extract the geometry (assuming it's a Point geometry, for polygons, use centroids or bounds)
    if row['geometry'].geom_type == 'Point':
        location = [row['geometry'].y, row['geometry'].x]  # Get latitude and longitude from Point
    elif row['geometry'].geom_type == 'Polygon':
        location = [row['geometry'].centroid.y, row['geometry'].centroid.x]  # Use the centroid for polygons

    print(stn_code)
    # Get the marker color based on the MRT line
    marker_color = get_line_color(stn_code)

    # Add a marker with the station code as tooltip and a custom popup
    folium.Marker(
        location=location,
        popup=folium.Popup(popup_text, max_width=300),
        tooltip=f"Station Code: {stn_code}",  # Tooltip will show the station code
        icon=folium.Icon(color=marker_color)
    ).add_to(mrt_map)

# Display the map inline in Jupyter Notebook
mrt_map

    

NS16
NE15
SE3
PE4
SW5
NE17/PTC
DT23
SW3
DT22
DT13
SW1
DT18
NE4/DT19
EW2/DT32
DT31
nan
nan
nan
nan
NS1/EW24
NS1/EW24
nan
nan
CC8
PW6
PW5
NS12
nan
nan
DT30
DT29
EW27
EW26
NS2
BP2
BP5
NS5
NS3
DT6
DT27
CC10/DT26
CC10/DT26
DT25
CC28
NE9
EW21/CC22
EW21/CC22
EW11
NS14
CC17/TE9
CC17/TE9
TE1
nan
nan
NS16
nan
nan
EW1
nan
EW1
TE7
nan
NS17/CC15
nan
TE29
nan
EW16/NE3/TE17
TE22
TE3
NS15
TE12
EW27
nan
CC26
DT10/TE11
CC19/DT9
CC19/DT9
EW8/CC9
nan
TE6
EW22
DT1/BP6
NS9
TE2
DT33
NS28
NE11
CG2
CG1/DT35
nan
EW17
DT5
EW6
CC7
NS19
NS18
CC12
NS17/CC15
NE10
CC16
nan
NS20
NS7
NS8
NS9
TE2
NS10
NS11
NS13
NS27/CE2/TE20
NS25/EW13
nan
NS23
NS21/DT11
NE5
nan
nan
nan
NS27/CE2/TE20
CC4/DT15
NS27/CE2/TE20
NS27/CE2/TE20
nan
SE4
CC6
CC5
NE8
CC21
CC24
nan
EW20
CC27
NE1/CC29
NE1/CC29
EW29
EW28
NE16/STC
PW3
SW7
SW4
PE3
NE12/CC13
SW8
CC20
NE12/CC13
NE14
PE1
SE1
PE6
SW6
PE2
PW7
PE5
TE4
DT10/TE11
EW9
nan
CG1/DT35
EW18
DT8
TE19
CC2
NE7/DT12
NE7/DT12
nan
nan
EW25
EW16/NE3/TE17
TE8
EW8/CC9
NS22/TE14
TE15
NS24/NE6/CC1
NS24/NE6/CC1


In [31]:
# todo: extract info from headers
mrt_map

In [ ]:
# Read the train stat

In [6]:
Tel = RailStations[RailStations['STN_LINE'] == 'Thomson East Coast']
Tel
# ax = PlanningArea.plot(color='white', edgecolor='black', figsize=(9, 12))
# # BusStops.plot(ax=ax , color="blue", markersize = 0.5)
# # Stations.plot(ax=ax, color='red', markersize=10)
# Tel.plot(ax=ax, color='brown', alpha=0.5)

KeyError: 'STN_LINE'

In [ ]:
# def get_train_service_lines(stn_no):
#     mapping = {
#         'NS': 'North-South',
#         'EW': 'East-West',
#         'CG': 'East-West',
#         'NE': 'North-East',
#         'CC': 'Circle',
#         'CE': 'Circle',
#         'DT': 'Downtown',
#         'TE': 'Thomson East Coast',
#         'BP': 'Bukit Panjang LRT',
#         'ST': 'Sengkang LRT', # STC is Sengkang LRT
#         'SE': 'Sengkang LRT',
#         'SW': 'Sengkang LRT',
#         'PW': 'Punggol LRT',
#         'PE': 'Punggol LRT',
#         'PT': 'Punggol LRT',  # PTC is Punggol LRT
#     }
#     line = mapping.get(stn_no[:2], 'Unknown')
#     if line == "Unknown":
#         print(f"Unknown train service line for station number: {stn_no}")
#     return line

# Stations['STN_LINE'] = Stations['STN_NO'].map(get_train_service_lines)
# Stations.head()